# Demo Flow with Human In the Loop

In [1]:
import gait as G
import ipywidgets as widgets
from IPython.display import display

In [2]:
class Head(G.Node):
    """Head of the flow"""

    def exec(self, _) -> str:
        print("Head")
        return G.Node.DEFAULT

In [3]:
class HIL(G.Node):
    """Human In the Loop"""

    def exec(self, sp: G.Scratchpad) -> str:
        # Check if a human input is present in the scratchpad.
        if sp["human_input"] is None:
            print("Need human input...")
            # Here we make sure to return nothing to stop the flow.
            return
        else:
            # Human input exists, continue the flow.
            print("We have human input now, we can continue...")
            return G.Node.DEFAULT

In [4]:
class Last(G.Node):
    """The last node in the flow"""

    def __init__(self, text_input) -> None:
        super().__init__()
        self.text_input = text_input

    def exec(self, sp: G.Scratchpad) -> str:
        self.text_input.value = str(sp["human_input"])
        return G.Node.DEFAULT

## Create a text widget

In [5]:
text_input = widgets.Text(
    value="",  # Initial value (empty in this case)
    placeholder="Input",  # Placeholder text
    description="Human:",  # Label for the input field
    disabled=True,  # Allow editing (set to False if you want to disable it)
)

In [6]:
## Define the node sequence or the graph.

In [7]:
(head := Head()) >> HIL() >> Last(text_input);

## Define a flow.

In [8]:
flow = G.Flow(head)
flow.display_markdown()

```mermaid
flowchart LR
5327131472["Head"] --> 5327137552["HIL"]
5327137552["HIL"] --> 5336523856["Last"]
```

## Create widget for human input.

In [9]:
slider = widgets.IntSlider(
    min=0,
    max=100,
    step=1,
    description="Value:",
    value=50,  # Initial value
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
)

## Let's start the flow.

In [10]:
flow();

Head
Need human input...


## Check for human input - SHOULD be missing.

In [11]:
flow["human_input"]

## Dump the state.

In [12]:
state = flow.dump_state()

In [13]:
display(text_input)
display(slider)

Text(value='', description='Human:', disabled=True, placeholder='Input')

IntSlider(value=50, continuous_update=False, description='Value:')

## Set the human input in the state scratchpad.

In [16]:
state.scratchpad["human_input"] = slider.value

## You can continue with new Flow with the dumped state.

Note: how to the text widget reflects the slider value.

In [17]:
flow = G.Flow(head, observer=G.FlowObserverConsole())
# Make sure to load the dumped state.
flow.load_state(state)
flow();

Flow started...
HIL started...
We have human input now, we can continue...
HIL ended.
Last started...
Last ended.
Flow ended.
